# Chatbots and Message history

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
model = ChatOpenAI(model='gpt-3.5-turbo')

## Message history

### Basic idea

In [3]:
from langchain_core.messages import HumanMessage, AIMessage

response = model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

response.content

'Your name is Bob.'

### With ChatMessageHistory

In [2]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [3]:
config = {"configurable": {"session_id": "session1"}}

In [6]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Sudip")],
    config=config,
)
response.content

'Hello Sudip! How can I assist you today?'

**Test if the model can remember my previous chat**

In [7]:
config = {"configurable": {"session_id": "session1"}}
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config,
)
response.content

'Your name is Sudip.'

**Now if provided new session id, model can't access my chat history**

In [8]:
config = {"configurable": {"session_id": "session2"}}
response = with_message_history.invoke(
    [HumanMessage(content="what is my name?")],
    config=config,
)
response.content

"I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."

## Using prompt template with message history

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all the questions to the best of your ability. In the final output first print your ourput in english and then translate it. Output must be in the format mentioned in the triple backticks. Output Fotmat: ```your output.  Translation: Translation of your outpput into {language}``` "
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

parser = StrOutputParser()
chain = prompt | model | parser

In [30]:
response = chain.invoke({
    "messages": [HumanMessage(content="hi! I'm sudip")],
    "language": "Italian",
})

response

'Hello Sudip! How can I assist you today?  \nTranslation: Ciao Sudip! Come posso aiutarti oggi?'

**Wrapping the model with Message History object**

In [35]:
with_message_history = RunnableWithMessageHistory(
    chain, 
    get_session_history,
    input_messages_key="messages"
)

In [36]:
config = {"configurable": {"session_id": "session3"}}

In [39]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="hi! I am Sudip.")],
        "language": "Italian",
    },
    config=config,
)
response

'Hello Sudip! How can I assist you today?  \nTranslation: Ciao Sudip! Come posso aiutarti oggi?'

In [40]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my name?")],
        "language": "Italian",
    },
    config=config,
)
response

'Your name is Sudip.  \nTranslation: Il tuo nome è Sudip.  '

## Managing increasing conversation 

Importantly, you will want to do this BEFORE the prompt template but AFTER you load previous messages from Message History.

In [46]:
from langchain_core.messages import SystemMessage, AIMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

invoking trimmer in messages will reduce the number of messages passes

In [47]:
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant"),
 HumanMessage(content='whats 2 + 2'),
 AIMessage(content='4'),
 HumanMessage(content='thanks'),
 AIMessage(content='no problem!'),
 HumanMessage(content='having fun?'),
 AIMessage(content='yes!')]

To use it in our chain, we just need to run the trimmer before we pass the messages input to our prompt.

### without message history wrapper

In [48]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | model
)

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what's my name?")],
        "language": "italian"
    }
)

response.content

"I'm sorry, I don't know your name."

In [49]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'you asked "what\'s 2 + 2"'

### with message history wrapper

In [50]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

config = {"configurable": {"session_id": "session4"}}

In [51]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "Italian",
    },
    config=config,
)
response.content

"I'm sorry, I do not have access to personal information."

For testing we have added more messages so the message where I had mentioned my name is trimmed

# Streaming

In [55]:
config = {"configurable": {"session_id": "session5"}}

for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="hi! I am sudip. I am a Computer Science Undergrad.")],
        "language": "italian",
    },
    config=config,
):
    print(r.content, end=" ")

 Hello  Sud ip !  It 's  nice  to  meet  you .  How  can  I  assist  you  today ?
 ``` Hello  Sud ip !  È  un  pi ac ere  con os cert i .  Come  pos so  ai ut arti  oggi ? ```  